# Competition

[Mercari Price Suggestion Challenge](https://www.kaggle.com/c/mercari-price-suggestion-challenge)

# Overview

メルカリに出品されている商品データ（カテゴリーやブランドなど）と価格をもとに<br>
商品データから価格を予測するモデルを作成する。

# Module

In [ ]:
import gc
import sys
import optuna
import datetime
import warnings
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Unfreeze

Kaggleで用意されたデータが、7z形式になっているため<br>
解凍の処理から行っていく。

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

下記は、SettingsのInternetをONにして実行しないと、エラーになる。

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

# Datasets

In [ ]:
train = pd.read_csv('train.tsv', sep='\t')

print(train.shape)
train.head(10)

In [ ]:
pd.DataFrame([['train_id', '商品ID'],
              ['name', '商品名'],
              ['item_condition_id', '商品の状態'],
              ['category_name', 'カテゴリー名'],
              ['brand_name', 'ブランド名'],
              ['price', '価格（単位：ドル）'],
              ['shipping', '送料が売手負担かどうか'],['item_description', '商品説明']],
              columns=['カラム', '意味'])

testデータは、test.tsvとtest_stg2.tsvがある。<br>
submitのときには、test_stg2を使用する。

In [ ]:
test_dummy = pd.read_csv('test.tsv', sep='\t')

print(test_dummy.shape)
test_dummy.head(10)

In [ ]:
pd.DataFrame([['test_id', '商品ID'],
              ['name', '商品名'],
              ['item_condition_id', '商品の状態'],
              ['category_name', 'カテゴリー名'],
              ['brand_name', 'ブランド名'],
              ['shipping', '送料が売手負担かどうか'],['item_description', '商品説明']],
              columns=['カラム', '意味'])

In [ ]:
test = pd.read_csv('test_stg2.tsv', sep='\t')

print(test.shape)
test.head(10)

In [ ]:
pd.DataFrame([['test_id', '商品ID'],
              ['name', '商品名'],
              ['item_condition_id', '商品の状態'],
              ['category_name', 'カテゴリー名'],
              ['brand_name', 'ブランド名'],
              ['shipping', '送料が売手負担かどうか'],['item_description', '商品説明']],
              columns=['カラム', '意味'])

In [ ]:
del test_dummy
gc.collect()

# Analytics

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
train.isnull().sum()

In [ ]:
train.corr()

In [ ]:
sns.heatmap(train.corr(), annot=True, fmt='.2f')

In [ ]:
train.describe()

# Visualization

In [ ]:
sns.set()

In [ ]:
train = train.drop(train[(train['price'] >= 200)].index, axis=0)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.histplot(train['price'], bins=50, ax=ax)
plt.show()

In [ ]:
train['log_price'] = np.log1p(train['price'])

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sns.histplot(train['log_price'], bins=50, ax=ax)
plt.show()

In [ ]:
train = train.drop('log_price', axis=1)
gc.collect()

In [ ]:
fig = sns.FacetGrid(train, hue='shipping', height=4, aspect=2)
fig.map(sns.kdeplot, 'price', shade=True)
plt.legend()

In [ ]:
fig = sns.FacetGrid(train, hue='item_condition_id', height=4, aspect=2)
fig.map(sns.kdeplot, 'price', shade=True)
plt.legend()

# Preprocess

In [ ]:
train_row = train.shape[0]
test_id = test['test_id']

In [ ]:
target = np.log1p(train['price'])
train = train[[col for col in train.columns if col != 'price']]

In [ ]:
train = train.drop('train_id', axis=1)
test = test.drop('test_id', axis=1)

In [ ]:
X = pd.concat([train, test], axis=0)

print(X.shape)
X.head()

In [ ]:
X.isnull().sum()

文字列を数値に変換するには、いくつかの方法がある。

・https://linus-mk.hatenablog.com/entry/pandas-unique-integer-id

また、数値変換を統一するために、trainとtestを結合して、まとめて実行する必要がある。

In [ ]:
X['category_name'] = X['category_name'].fillna(value='missing')
X['item_description'] = X['item_description'].fillna(value='missing')
X['name'] = X['name'].fillna(value='missing')
X['brand_name'] = X['brand_name'].fillna(value='missing')

In [ ]:
X['category_name'] = X['category_name'].astype('category')
X['item_description'] = X['item_description'].astype('category')
X['name'] = X['name'].astype('category')
X['brand_name'] = X['brand_name'].astype('category')

In [ ]:
X['name'] = X['name'].cat.codes
X['category_name'] = X['category_name'].cat.codes
X['brand_name'] = X['brand_name'].cat.codes
X['item_description'] = X['item_description'].cat.codes

In [ ]:
train = X[:train_row]
test = X[train_row:]

In [ ]:
del train_row
gc.collect()

In [ ]:
train = pd.concat([train, target], axis=1)

print(train.shape)
train.head()

In [ ]:
df_group = train.groupby('category_name')['price']

In [ ]:
category_mean = df_group.mean()
category_median = df_group.median()
category_std = df_group.std()
category_min = df_group.min()
category_max = df_group.max()

In [ ]:
train['category_mean'] = train['category_name'].map(category_mean)
train['category_median'] = train['category_name'].map(category_median)
train['category_std'] = train['category_name'].map(category_std)
train['category_min'] = train['category_name'].map(category_min)
train['category_matrain'] = train['category_name'].map(category_max)

In [ ]:
del df_group
gc.collect()

In [ ]:
train = train[[col for col in train.columns if col != 'price']]

In [ ]:
train.to_pickle('train.pickle')

print(train.shape)
train.head()

In [ ]:
print(target.shape)
target.head()

# Modeling

In [ ]:
'''
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=666)

def create_model(trial):
    num_leaves = trial.suggest_int('num_leaves', 2, 30)
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0001, 0.99)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    min_child_samples = trial.suggest_int('min_child_samples', 100, 1200)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 5, 90)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 7)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.0001, 1.0)
    feature_fraction = trial.suggest_uniform('feature_fraction', 0.0001, 1.0)
    subsample = trial.suggest_uniform('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.1, 1.0)
    
    model = lgb.LGBMRegressor(
        num_leaves=num_leaves,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        min_data_in_leaf=min_data_in_leaf,
        bagging_freq=bagging_freq,
        bagging_fraction=bagging_fraction,
        feature_fraction=feature_fraction,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        metric='rsme',
        random_state=666)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    rsme = np.sqrt(mean_squared_error(y_pred, y_val))
    return rsme

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=40)
params = study.best_params
print(params)
'''

In [ ]:
params = {'num_leaves': 27,
          'n_estimators': 246,
          'learning_rate': 0.5891085520981525,
          'max_depth': 8,
          'min_child_samples': 736,
          'min_data_in_leaf': 80,
          'bagging_freq': 3,
          'bagging_fraction': 0.7976794164483426,
          'feature_fraction': 0.7716993533001292,
          'subsample': 0.3585480926624078,
          'colsample_bytree': 0.28978876816462784,
          'random_state': 666}

In [ ]:
cls = lgb.LGBMRegressor(**params)
cls.fit(train, target)

In [ ]:
lgb.plot_importance(cls)
plt.show()

In [ ]:
del train, target, params
gc.collect()

# Submit

In [ ]:
test['category_mean'] = test['category_name'].map(category_mean)
test['category_median'] = test['category_name'].map(category_median)
test['category_std'] = test['category_name'].map(category_std)
test['category_min'] = test['category_name'].map(category_min)
test['category_max'] = test['category_name'].map(category_max)

In [ ]:
test.to_pickle('test.pickle')

print(test.shape)
test.head()

In [ ]:
prediction = np.expm1(cls.predict(test))

In [ ]:
del test
gc.collect()

In [ ]:
submit = pd.DataFrame(test_id, columns=['test_id'])
submit['price'] = prediction

submit.head(10)

In [ ]:
del test_id, prediction
gc.collect()

Submissionファイルは、/kaggle下ではなく、/kaggle/working下に作成する。<br>
また、submissionのCSVファイルについて、index=Falseにする。

In [ ]:
submit.to_csv('./submission.csv', index=False)